In [106]:
import geopandas as gpd
import fiona
import osmnx as ox
import folium
from fiona.model import to_dict
from shapely.geometry import shape
from branca.colormap import LinearColormap
import numpy as np
import networkx as nx
import pandas as pd

import matplotlib.pyplot as plt
from scipy.spatial import Voronoi,voronoi_plot_2d
from geovoronoi import voronoi_regions_from_coords, points_to_coords
import random
from shapely.geometry import Polygon, Point

from skmob.tessellation.tilers import tiler
from skmob.utils.plot import plot_gdf
import requests

In [58]:
def Leuven_map():
    point = (50.849738563227824, 4.644786576420597)
    m = folium.Map(location=point, zoom_start = 12)
    return m

# DATA

In [2]:
# ciclabile/strada
cyclic = gpd.read_file('https://storageaccount11111111.blob.core.windows.net/container1/Leuven/cycling_data/cycling_network.gpkg')
cyclic = cyclic.to_crs(epsg=4326)

In [5]:
#info socio-dem
socio_dem_data = pd.read_excel('https://storageaccount11111111.blob.core.windows.net/container1/Leuven/socio_demographic_data/socio_demographic_data_leuven_2019.xlsx')
#drop nan zone
socio_dem_data.drop(np.where(socio_dem_data['CODSEC'].isna())[0][0], axis = 0 ,inplace=True)

In [16]:
#zone (enorme)
zones = gpd.read_file('https://storageaccount11111111.blob.core.windows.net/container1/Leuven/od_data_leuven/OD_zones_leuven.gpkg')
zones = zones.to_crs('EPSG:4326')
zones_leuven = zones[zones['VREGIO_L'] == 'LEUVEN']

In [97]:
#ciclabili per le zone selezionate
ind = set()
for i in range(len(cyclic)):
    z = zones_leuven[zones_leuven['NIS_GEM'] == 24062]['geometry']
    s = np.where(z.intersects(cyclic['geometry'][i]))[0]
    if len(s) > 0:
        ind.add(i)

In [101]:
for i in ind:
    folium.GeoJson(cyclic['geometry'][i]).add_to(m)

In [105]:
leuven_cyclic = cyclic.iloc[list(ind), :]

In [107]:
# off_street_parking
url = 'https://storageaccount11111111.blob.core.windows.net/container1/Leuven/parking_data_leuven/off_street_parking_supply_Leuven.json'
response = requests.get(url)
json_data = response.json()

results = json_data['JSON']['results']
normalized_data = pd.json_normalize(results)

df = pd.DataFrame(normalized_data)

In [ ]:
# estraggo info dalla colonna bullets 
spots = []
cameras = []
accessibility = []
lift = []
height = []


for row in df['bullets']:
    spot_number = None
    camera_label = None
    accessibility_label = None
    lift_label = None
    height_label = None
    
    for item in row:
        label = item['label']
        if 'spots' in item['icon']:
            spot_number = int(''.join(filter(str.isdigit, label)))
        elif 'camera' in item['icon']:
            camera_label = label
        elif 'accessibility' in item['icon']:
            accessibility_label = label
        elif 'lift' in item['icon']:
            lift_label = label
        elif 'height' in item['icon']:
            height_label = label
    

    spots.append(spot_number)
    cameras.append(camera_label)
    accessibility.append(accessibility_label)
    lift.append(lift_label)
    height.append(height_label)

df['spots'] = spots
df['cameras'] = cameras
df['accessibility'] = accessibility
df['lift'] = lift
df['height'] = height

df = df.drop('bullets', axis=1)

In [141]:
geometry = [Point(xy) for xy in zip(df['coords.lng'], df['coords.lat'])]
gdf = gpd.GeoDataFrame(df, geometry=geometry, crs="EPSG:4326")

columns_subset = ['pricing_type', 'free_bus', 'id', 'subtype', 'spots', 'lift', 'geometry']
gdf = gdf[columns_subset]
mapping = {'None': 0, 'Parking met lift': 1, True: 1, False:0}
gdf['lift'] = gdf['lift'].replace(mapping)
gdf['free_bus'] = gdf['free_bus'].replace(mapping)
gdf['lift'] = gdf['lift'].fillna(0)
gdf.shape

(43, 7)

## calcolo distanze tra :

In [135]:
import pyproj
from shapely.ops import transform

In [149]:
utm = leuven_cyclic.estimate_utm_crs()
project = pyproj.Transformer.from_crs('4326', utm)

### Parcheggi

In [180]:
transformed_parkings = []
for point in gdf['geometry']:
    transformed_parkings.append(transform(project.transform, point))

for candidate in leuven_cyclic['geometry']:
    g2 = transform(project.transform, candidate)  
    print(g2.distance(transformed_parkings))
    #print(np.min(g2.interpolation(transformed_parkings)))

[5808.78317612 6498.96115617 6584.63840773 6097.83892227 6063.74626164
 6345.61063777 5786.24897715 6076.07173346 5407.50537083 5983.43548225
 4082.45695654 6309.10344926 7208.63821733 8445.90075834 7114.99372652
 7706.28683121 6687.24067263 8226.69120231 6029.89067924 5913.54892681
 5926.96661758 5649.27229762 6433.18450227 6420.87386876 6489.05813196
 4980.52859899 6263.45998123 6546.75798212 6718.53403441 6809.28183835
 6886.09785109 5055.59185185 5467.32369018 5690.60005297 6673.70448613
 5642.14971958 4819.39732219 5691.43056565 5516.59239354 4965.58396633
 6043.84612677 6840.00266402 6935.85814077]


AttributeError: 'LineString' object has no attribute 'interpolation'